![Rampart Logo](../images/logo.png)

Auge is an image classification model. Its main target's to recognize common photos to determine a few flat publication's features. Each image belongs to a specific realty, having a bunch of photo & panorama recognized **twinkle** can better predict apartments' order.

## I/O
Required data consists of two parts + remote image CDN:
- Training references lie in `../data/auge/training.csv` ;
- Testing links lie in `../data/auge/testing.csv` .

Currently the DB containes images of two types: simple photos & wide (360 deg) panoramas. Until a stable classification model panoramas should be omitted. Final classifier must be stored into `../models/auge.pth` .

## Classes
- `luxury` is a flat with rich furniture, huge rooms, chandeliers, fireplaces, etc.
![Luxury 1](../images/luxury1.webp)
![Luxury 2](../images/luxury2.webp)
![Luxury 3](../images/luxury3.webp)
- `comfort` is the most suitable for an ordinary citizen apartments. Clean, neat, sometimes minimalistic, average area, qualitive furniture, etc.
![Comfort 1](../images/comfort1.webp)
![Comfort 2](../images/comfort2.webp)
![Comfort 3](../images/comfort3.webp)
- `construction` is a flat without a finished design. No doors, floor, supplies, wallpapers, ceiling, furniture, etc. Typically, new buildings contain these apartments.
![Construction 1](../images/construction1.webp)
![Construction 2](../images/construction2.webp)
![Construction 3](../images/construction3.webp)
- `junk` is an old flat image. Probably, the whole apartments should belong to a dormitory, Khrushchevka or gostinka.
![Junk 1](../images/junk1.webp)
![Junk 2](../images/junk2.webp)
![Junk 3](../images/junk3.webp)
- `excess` is the trash category. Actually, all exterior photos, outlines & posters lie here.
![Excess 1](../images/excess1.webp)
![Excess 2](../images/excess2.webp)
![Excess 3](../images/excess3.webp)

In [1]:
%matplotlib inline

In [2]:
from plotly.graph_objs import Pie, Figure, Scatter
from plotly.figure_factory import create_annotated_heatmap
from plotly.offline import init_notebook_mode, iplot
from re import match
from numpy import arange, trace, sum
from glob import glob
from pandas import DataFrame
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import Compose, ToTensor, Normalize, Resize
from torch.nn import Module, Conv2d, Linear, ReLU, CrossEntropyLoss, Sequential
from torch.optim import Adam
from torch import no_grad, save, max, load, zeros, cat, long
from PIL.Image import open
from time import time

In [3]:
init_notebook_mode(True)

In [4]:
def parse(path):
    result = match(r'^.*/(\w+)\.(\w+)\.webp$', path)
    return ('unknown', 'unknown') if not result else result.groups()    

In [5]:
labels = ['luxury', 'comfort', 'junk', 'construction', 'excess']
mappings = {l: i for i, l in enumerate(labels)}

In [6]:
images = DataFrame({'path': glob('../data/auge/images/*.webp')})
images['label'] = images['path'].apply(lambda p: mappings[parse(p)[1]])

In [7]:
counts = images['label'].value_counts().sort_index()
iplot(
    Figure()
    .add_trace(Pie(labels=[labels[i] for i in counts.index], values=counts.values))
    .update_layout(margin={'t': 30, 'r': 0, 'b': 0, 'l': 0})
)

In [8]:
class AugeDataset(Dataset):
    def __init__(self, data):
        self._data = data.values
        self._transforms = Compose(
            [
                ToTensor(),
                Resize((230, 310)),
                Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225], True)
            ]
        )

    def __len__(self):
        return len(self._data)

    def __getitem__(self, index):
        path, label = self._data[index]
        return self._transforms(open(path)), label

In [9]:
training_images, testing_images = train_test_split(images, test_size=0.2, stratify=images['label'])
training_images, validation_images = train_test_split(
    training_images,
    test_size=0.2,
    stratify=training_images['label']
)
batch_size = 4
training_loader = DataLoader(AugeDataset(training_images), batch_size, True)
validation_loader = DataLoader(AugeDataset(validation_images), batch_size)
testing_loader = DataLoader(AugeDataset(testing_images), batch_size)

In [10]:
class View(Module):
    def __init__(self, *shape):
        super().__init__()
        self._shape = shape

    def forward(self, x):
        return x.view(*self._shape)

In [11]:
class Auge(Module):
    def __init__(self):
        super().__init__()
        self._sequential = Sequential(
            Conv2d(3, 5, 11, 2),
            ReLU(),
            Conv2d(5, 10, 9, 2),
            ReLU(),
            Conv2d(10, 15, 3),
            ReLU(),
            View(-1, 50715),
            Linear(50715, 2048),
            ReLU(),
            Linear(2048, 256),
            ReLU(),
            Linear(256, 5)
        )

    def forward(self, x):
        return self._sequential(x)

In [ ]:
auge = Auge()
criterion = CrossEntropyLoss()
optimizer = Adam(auge.parameters())
training_losses, validation_losses = [], []
epoch_number = 5
for epoch in range(1, epoch_number + 1):
    start = time()
    training_loss, validation_loss = 0.0, 0.0
    auge.train()
    for samples, targets in training_loader:
        optimizer.zero_grad()
        loss = criterion(auge(samples), targets)
        loss.backward()
        optimizer.step()
        training_loss += loss.item() * samples.size(0)
    auge.eval()
    for samples, targets in validation_loader:
        validation_loss += criterion(auge(samples), targets).item() * samples.size(0)
    end = time()
    training_loss /= len(training_loader.sampler)
    validation_loss /= len(validation_loader.sampler)
    training_losses.append(training_loss)
    validation_losses.append(validation_loss)
    print(
        f'Epoch: {epoch:2}  '
        f'Training loss: {training_loss:.6f}  '
        f'Validation loss: {validation_loss:.6f}  '
        f'Duration: {end - start:.0f}s'
    )
save(auge.state_dict(), '../models/auge.pth')
iplot(
    Figure()
    .add_trace(Scatter(x=arange(1, epoch_number + 1), y=training_losses, name='training'))
    .add_trace(Scatter(x=arange(1, epoch_number + 1), y=validation_losses, name='validation'))
    .update_layout(margin={'t': 30, 'r': 0, 'b': 0, 'l': 0})
)

In [13]:
auge = Auge()
auge.load_state_dict(load('../models/auge.pth'))
auge.eval()
actual, expected = zeros(0, dtype=long), zeros(0, dtype=long)
with no_grad():
    for samples, targets in testing_loader:
        actual = cat([actual, max(auge(samples), 1)[1].view(-1)])
        expected = cat([expected, targets.view(-1)])
matrix = confusion_matrix(expected, actual)
iplot(
    create_annotated_heatmap(z=matrix, x=labels, y=labels, hoverinfo='skip')
    .update_xaxes(title_text='Expected')
    .update_yaxes(title_text='Actual', autorange='reversed')
    .update_layout(
        margin={'t': 100, 'r': 0, 'b': 0, 'l': 0},
        title=f'Accuracy: {trace(matrix) / sum(matrix) * 100:.2f}%'
    )
)

In [ ]:
# from cv2 import imread, IMREAD_COLOR, cvtColor, COLOR_BGR2RGB, IMREAD_GRAYSCALE, imwrite, inpaint, INPAINT_TELEA
# def d3(matrix):
#     return stack((matrix, matrix, matrix), 2)
# url = '../data/auge/images/d379f0f7ed01b7290a3a905e9daba1d1276ef7d3.comfort.webp'
# gray = imread(url, IMREAD_GRAYSCALE)
# mask = copy(gray)
# mask[341:443, 125:272] = 255
# mask[:341, :] = 0
# mask[443:, :] = 0
# mask[:, :125] = 0
# mask[:, 272:] = 0
# for x in range(125, 165):
#     mask[:382 + 126 - x, x] = 0
# for x in range(210, 272):
#     mask[:x + 130, x] = 0
# name, label = parse(url)
# iplot(
#     make_subplots(cols=2, horizontal_spacing=0.03)
#     .add_trace(Image(z=d3(gray), name=''), 1, 1)
#     .add_trace(Image(z=d3(mask), name=''), 1, 2)
#     .update_layout(title=f'{name} - {label}', margin={'t': 40, 'r': 0, 'b': 0, 'l': 0}, height=380)
#     .update_xaxes(showticklabels=False)
#     .update_yaxes(showticklabels=False)
# )
# imwrite('../data/auge/masks/olymp.jpeg', mask)
# url = '../data/auge/images/b7bb98d1f53cbc8614fad2e5926975a13904e135.construction.webp'
# bgr = imread(url, IMREAD_COLOR)
# name, label = parse(url)
# iplot(
#     make_subplots(cols=2, horizontal_spacing=0.03)
#     .add_trace(Image(z=cvtColor(bgr, COLOR_BGR2RGB), name=''), 1, 1)
#     .add_trace(
#         Image(
#             z=cvtColor(
#                 inpaint(bgr, imread('../data/auge/masks/domria.jpeg', IMREAD_GRAYSCALE), 4, INPAINT_NS),
#                 COLOR_BGR2RGB
#             ),
#             name=''
#         ),
#         1,
#         2
#     )
#     .update_layout(title=f'{name} - {label}', margin={'t': 40, 'r': 0, 'b': 0, 'l': 0}, height=380)
#     .update_xaxes(showticklabels=False)
#     .update_yaxes(showticklabels=False)
# )